# Importing data into Neo4j from Python

Using the Neo4j Python driver, we are importing some data from an `edgelist` file. All nodes are assigned the `NodeB` label.

In [1]:
from collections import defaultdict
from neo4j import GraphDatabase

In [2]:
def process_row(tx, row):
    id1, id2 = row.split(" ")
    tx.run("MERGE (n1:NodeB {id: $id1}) "
           "MERGE (n2:NodeB {id: $id2})"
           "CREATE (n1)-[:LINK]->(n2)",
           id1=int(id1), id2=int(id2),
        )

In [3]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "<PASSWORD>"))

In [4]:
!ls ../data

graph_A.edgelist  graph_B.edgelist


In [15]:
with open("../data/graph_B.edgelist", "r") as f:
    with driver.session() as s:
        for row in f:
            s.write_transaction(process_row, row)

In [5]:
with driver.session() as s:
    result = s.run("MATCH (n) RETURN n")
    data = defaultdict(int)
    for r in result:
        for l in r["n"].labels:
            data[l] += 1
data

defaultdict(int, {'NodeA': 2001, 'NodeB': 800, 'NodeC': 5})